In [0]:
%pip install yfinance

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 53.8 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=82aae3ffc5a99b8207b6ad9c37c9e6f9bafdb9b23133f21001064ecba755820d
  Stored in directory: /home/spark-8bdcf31a-c660-4fa8-9394-8f/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.3-cp312-cp312-linux_x86_64.whl size=942269 sha256=cc3e532c86

In [0]:
import yfinance as yf
import pandas as pd
import numpy as np

In [0]:
# Configure authentication 
spark.conf.set("fs.azure.account.auth.type.stocksdl.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.stocksdl.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.stocksdl.dfs.core.windows.net", "CLIENT-ID")
spark.conf.set("fs.azure.account.oauth2.client.secret.stocksdl.dfs.core.windows.net", "SECRET-KEY")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.stocksdl.dfs.core.windows.net", "https://login.microsoftonline.com/TENANT-ID/oauth2/token")


In [0]:
# Importing Stock Tickers
ticker_aapl = "AAPL"
tic_aapl = yf.Ticker(ticker_aapl)

ticker_mrna = "MRNA"
tic_mrna = yf.Ticker(ticker_mrna)

ticker_nvda = "NVDA"
tic_nvda = yf.Ticker(ticker_nvda)

ticker_ocg = "OCG"
tic_ocg = yf.Ticker(ticker_ocg)

ticker_tsla = "TSLA"
tic_tsla = yf.Ticker(ticker_tsla)

In [0]:
# Reading Historical Data
hist_aapl = tic_aapl.history(period="10y")
hist_aapl.reset_index(inplace=True)

hist_mrna = tic_mrna.history(period="10y")
hist_mrna.reset_index(inplace=True)

hist_nvda = tic_nvda.history(period="10y")
hist_nvda.reset_index(inplace=True)

hist_ocg = tic_ocg.history(period="10y")
hist_ocg.reset_index(inplace=True)

hist_tsla = tic_tsla.history(period="10y")
hist_tsla.reset_index(inplace=True)

In [0]:
# print(hist_data.tail(100))
# print(todays.tail(100))

In [0]:
# Converting to Spark and Dropping unnecessary cols for 'HISTORICAL DATA'
spark_aapl = spark.createDataFrame(hist_aapl)
spark_aapl = spark_aapl.drop('Stock Splits', 'Dividends')

spark_mrna = spark.createDataFrame(hist_mrna)
spark_mrna = spark_mrna.drop('Stock Splits', 'Dividends')

spark_nvda = spark.createDataFrame(hist_nvda)
spark_nvda = spark_nvda.drop('Stock Splits', 'Dividends')

spark_ocg = spark.createDataFrame(hist_ocg)
spark_ocg = spark_ocg.drop('Stock Splits', 'Dividends')

spark_tsla = spark.createDataFrame(hist_tsla)
spark_tsla = spark_tsla.drop('Stock Splits', 'Dividends')

In [0]:
# hist_spark.display(limit=100)
# type(hist_spark)

In [0]:
# print(f"type(today_spark_aapl) == {type(today_spark_aapl)}")
# print(f"type(today_spark_mrna) == {type(today_spark_mrna)}")
# print(f"type(today_spark_nvda) == {type(today_spark_nvda)}")
# print(f"type(today_spark_ocg) == {type(today_spark_ocg)}")
# print(f"type(today_spark_tsla) == {type(today_spark_tsla)}")

# print(f"type(spark_aapl) == {type(spark_aapl)}")
# print(f"type(spark_mrna) == {type(spark_mrna)}")
# print(f"type(spark_nvda) == {type(spark_nvda)}")
# print(f"type(spark_ocg) == {type(spark_ocg)}")
# print(f"type(spark_tsla) == {type(spark_tsla)}")

In [0]:
# First writing in delta format historical data
# spark_aapl.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Apple/") #historical

# spark_mrna.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Moderna/") #historical 

# spark_nvda.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Nvidia/") #historical 

# spark_ocg.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Oriental/") #historical

# spark_tsla.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Tesla/") #historical

In [0]:
# Regular writing Historical data
from delta.tables import DeltaTable

#Apple
deltaTable_appl = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Apple/")
deltaTable_appl.alias("target").merge(
    spark_aapl.alias("source"),
   "target.Date = source.Date" 
).whenNotMatchedInsertAll().execute()

#Moderna
deltaTable_mrna = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Moderna/")
deltaTable_mrna.alias("target").merge(
    spark_mrna.alias("source"),
   "target.Date = source.Date" 
).whenNotMatchedInsertAll().execute()

#Nvidia
deltaTable_nvda = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Nvidia/")
deltaTable_nvda.alias("target").merge(
    spark_nvda.alias("source"),
   "target.Date = source.Date" 
).whenNotMatchedInsertAll().execute()

#Oriental
deltaTable_ocg = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Oriental/")
deltaTable_ocg.alias("target").merge(
    spark_ocg.alias("source"),
   "target.Date = source.Date" 
).whenNotMatchedInsertAll().execute()

#Tesla
deltaTable_tsla = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/historical-data/Tesla/")
deltaTable_tsla.alias("target").merge(
    spark_tsla.alias("source"),
   "target.Date = source.Date" 
).whenNotMatchedInsertAll().execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
dbutils.notebook.exit("Done")